In [1]:
import sys

sys.path.append('..')

In [2]:
import json
import pandas as pd
import spacy
from spacy import displacy
from tqdm import tqdm

from utils.read_data import read_jsonl
from NER.EntityExtractor import EntityExtractor

tqdm.pandas()

/opt/conda/envs/spacy_gpu/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def display_entities(text, entities, entity_type=None):
    nlp = spacy.blank("en")
    doc = nlp.make_doc(text)
    ents = []
    for entity in entities:
        if (entity["label"] == entity_type) or (entity_type is None):
            span_start = entity["start"]
            span_end = entity["end"]
            label = entity["label"]
            ent = doc.char_span(span_start, span_end, label=label)
            if ent is None:
                continue
            ents.append(ent)
    doc.ents = ents
    displacy.render(doc, style="ent", jupyter=True)

In [4]:
path = '../RE/assets/annotations_test.jsonl'
test = read_jsonl(path)

In [5]:
extractor = EntityExtractor(model='trf')

In [6]:
test.shape

(57, 6)

In [7]:
test['predictions'] = test.text.progress_apply(lambda x: extractor.get_predictions([x], log=False))

100%|██████████| 57/57 [00:01<00:00, 30.37it/s]


In [8]:
test['exact_preds'] = test.apply(lambda x: x['spans'] == x['predictions'][0], axis=1)

In [9]:
for i in range(10):
    print('')
    print(f'Example {i+1}')
    print('\nTruth')
    row = test[test.exact_preds==False].sample().iloc[0]
    display_entities(row.text, row.spans, None)
    print('\nPreds')
    display_entities(row.text, row.predictions[0], None)


Example 1

Truth



Preds



Example 2

Truth



Preds



Example 3

Truth


...


Preds



Example 9

Truth



Preds



Example 10

Truth



Preds
